In [1]:
import neo4j as neo

In [2]:
url = 'bolt://localhost:7687'
auth = ('neo4j', 'neo4j_')

In [3]:
def _run_query(tx, query, **args):
    """
    Run one query statement in transaction, and return result
    """
    result = tx.run(query, **args)
    return result
    
def _run_transaction(statement_list):
    # Make neo4j driver object
    driver = neo.GraphDatabase.driver(url, auth=auth)
    try:
        # Start neo4j session
        with driver.session() as session:
            for statement in statement_list:
                query = statement['query']
                args = statement.get('args', {})
                method = statement.get('method', 'READ')
                # Put each statement into transaction
                if methodod == 'WRITE':
                    result = session.write_transaction(_run_query, query, **args)
                else
                    result = session.read_transaction(_run_query, query, **args)
                    
                print(result.values())
                print([node for node in result.graph().nodes])
                print([r for r in result.graph().relationships])
    finally:
        driver.close()

In [23]:
"""
Create or merge object in neo4j
"""

statements = [
    {
        'query': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                 'RETURN person',
        'args': {
            'id': 1,
            'name': 'Alvin',
            'gender': 'M' 
        },
        'method': 'WRITE'
    },
    {
        'query': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                 'RETURN person',
        'args': {
            'id': 2,
            'name': 'Emma',
            'gender': 'F' 
        },
        'method': 'WRITE'
    }
]

_run_transaction(statements)

[[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>]]
[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>]
[]
[[<Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]]
[<Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[]


In [24]:
"""
Update exist object
"""

statements = [
    {
        'query': 'MATCH (person:Person)'
                 'WHERE person.id = {id} '
                 'SET person.birthday = {birthday} '
                 'RETURN person',
        'args': {
            'id': 1,
            'birthday': '1981-03-17'
        },
        'method': 'WRITE'
    },
    {
        'query': 'MATCH (person:Person)'
                 'WHERE person.id = {id} '
                 'SET person.birthday = {birthday} '
                 'RETURN person',
        'args': {
            'id': 2,
            'birthday': '1985-03-29'
        },
        'method': 'WRITE'
    }
]

_run_transaction(statements)

[[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>]]
[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>]
[]
[[<Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]]
[<Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[]


In [25]:
"""
Build relationship
"""

statements = [
    {
        'query': 'MATCH (p1:Person), (p2:Person)'
                 'WHERE p1.id = {id1} AND p2.id = {id2} '
                 'MERGE (p1)-[r:Married]->(p2) '
                 'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        },
        'method': 'WRITE'
    },
    {
        'query': 'MATCH (p1:Person), (p2:Person) '
                 'WHERE p1.id = {id1} AND p2.id = {id2} '
                 'MERGE (p2)-[r:Married]->(p1) '
                 'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        },
        'method': 'WRITE'
    }
]

_run_transaction(statements)

[[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>, 'Married']]
[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[]
[[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>, 'Married']]
[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[]


In [26]:
"""
Create object and build relationship
"""

statements = [
    {
        'query': 'MATCH (person:Person) '
                 'WHERE person.id = {person_id} '
                 'MERGE (cat:Cat { id: {cat_id}, name: {cat_name} }) '
                 'MERGE (person)-[r:Feed]->(cat) '
                 'RETURN person, cat, type(r)',
        'args': {
            'person_id': 1,
            'cat_id': 3,
            'cat_name': "Lily"
        },
        'method': 'WRITE'
    }
]

_run_transaction(statements)

[[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>, 'Feed']]
[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>]
[]


In [28]:
"""
Find person who feed cat Lily
"""

statements = [
    {
        'query': 'MATCH (person:Person)-[r:Feed]->(cat:Cat) '
                 'WHERE cat.name = {cat_name} '
                 'RETURN person, cat, type(r)',
        'args': {
            'cat_name': "Lily"
        }
    }
]

_run_transaction(statements)

[[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>, 'Feed']]
[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>]
[]


In [9]:
"""
Find persons who has relationship with cat Lily
"""

statements = [
    {
        'query': 'MATCH (person:Person)-[r*]->(cat:Cat) '
                 'WHERE cat.name = {cat_name} '
                 'RETURN DISTINCT person.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

_run_transaction(statements)

[['Alvin'], ['Emma']]
[]
[]


In [31]:
"""
Find who has relationship with cat Lily
"""

statements = [
    {
        'query': 'MATCH (p1:Person)-[:Married]-(p2:Person)-[r:Feed*]->(c:Cat) '
                 'WHERE c.name={cat_name} '
                 'RETURN DISTINCT p1.name',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

_run_transaction(statements)

[['Emma']]
[]
[]


In [30]:
"""
Find graph to cat Lily
"""

statements = [
    {
        'query': 'MATCH g = (person:Person)-[r*]->(cat:Cat) '
                 'WHERE cat.name = {cat_name} '
                 'RETURN g, length(g)',
        'args': {
            'cat_name': 'Lily'
        }
    }
]

_run_transaction(statements)

[[<Path start=<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> end=<Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=1>, 1], [<Path start=<Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}> end=<Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=2>, 2], [<Path start=<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> end=<Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}> size=3>, 3]]
[<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>, <Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>, <Node id=1 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>]
[<Relationship id=22 nodes=(<Node id=0 labels={'Person'} properties={'birthday': '1981-03-17', 'name':

In [29]:
"""
Find last children node in all relationship
"""

statements = [
    {
        'query': 'MATCH ()-[*]->(n) '
                 'WHERE NOT (n)-[]->() '
                 'RETURN DISTINCT n'
    }
]

_run_transaction(statements)

[[<Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>]]
[<Node id=20 labels={'Cat'} properties={'name': 'Lily', 'id': 3}>]
[]


In [163]:
"""
Match results
"""

driver = neo.GraphDatabase.driver(url, auth=auth)

try:
    
    def _query_last_node(tx):
        return tx.run('MATCH ()-[r*]->(n) '
                      'WHERE NOT (n)-[]->() '
                      'RETURN DISTINCT n')

    def _print_node(node):
        print('Node physical id is: {}'.format(node.id))
        print('Node label is: {}'.format(next(iter(node.labels))))
        print('Node property keys are: {}'.format(list(node.keys())))
        print('Node property values are: {}'.format(list(node.values())))
        print('Node property id is: {}'.format(node['id']))
        print('Node property name is: {}'.format(node.get('name')))

    with driver.session() as session:
        result = session.read_transaction(_query_last_node).values() # Get result values
        node = result[0][0]
        _print_node(node)

        
    def _print_relationship(relationships):
        for r in relationships:
            r = r[0][0]
            print('--->')
            print('\tRelationship physical id is: {}'.format(r.id))
            print('\tRelationship label is: {}'.format(r.type))
            print('\tRelationship start node id is [id={}, name={}]'.format(
                r.start_node['id'], r.start_node['name']))
            print('\tRelationship start node id is [id={}, name={}]'.format(
                r.end_node['id'], r.end_node['name']))

    def _query_relationship(tx):
        return tx.run('MATCH ()-[r*]->(n) '
                      'WHERE NOT (n)-[]->() '
                      'RETURN DISTINCT r, n')

    with driver.session() as session:
        result = session.read_transaction(_query_relationship).values() # Get result values
        _print_relationship(result)
        
finally:
    driver.close()

Node physical id is: 20
Node label is: Cat
Node property keys are: ['name', 'id']
Node property values are: ['Lily', 3]
Node property id is: 3
Node property name is: Lily
--->
	Relationship physical id is: 22
	Relationship label is: Feed
	Relationship start node id is [id=None, name=None]
	Relationship start node id is [id=3, name=Lily]
--->
	Relationship physical id is: 20
	Relationship label is: Married
	Relationship start node id is [id=None, name=None]
	Relationship start node id is [id=None, name=None]
--->
	Relationship physical id is: 0
	Relationship label is: Married
	Relationship start node id is [id=None, name=None]
	Relationship start node id is [id=None, name=None]


In [14]:
"""
Delete nodes with relationships
"""

statements = [
    {
        'query': 'MATCH ()-[mr:Married]->() '
                 'MATCH ()-[fr:Feed]->() '
                 'OPTIONAL MATCH (ps:Person) '
                 'OPTIONAL MATCH(cs:Cat) '
                 'DELETE mr, fr, ps, cs '
                 'RETURN mr, fr, ps, cs'
    }
]

_run_transaction(statements)

[[<Relationship id=22 nodes=(<Node id=0 labels=set() properties={}>, <Node id=20 labels=set() properties={}>) type='Married' properties={}>, <Relationship id=24 nodes=(<Node id=0 labels=set() properties={}>, <Node id=22 labels=set() properties={}>) type='Feed' properties={}>, <Node id=0 labels=set() properties={}>, <Node id=22 labels=set() properties={}>], [<Relationship id=22 nodes=(<Node id=0 labels=set() properties={}>, <Node id=20 labels=set() properties={}>) type='Married' properties={}>, <Relationship id=24 nodes=(<Node id=0 labels=set() properties={}>, <Node id=22 labels=set() properties={}>) type='Feed' properties={}>, <Node id=1 labels=set() properties={}>, <Node id=22 labels=set() properties={}>], [<Relationship id=22 nodes=(<Node id=0 labels=set() properties={}>, <Node id=20 labels=set() properties={}>) type='Married' properties={}>, <Relationship id=24 nodes=(<Node id=0 labels=set() properties={}>, <Node id=22 labels=set() properties={}>) type='Feed' properties={}>, <Node i

In [33]:
"""
Import from csv
"""

statements = [
    {
        'query': 'LOAD CSV WITH HEADERS FROM "file:///data.csv" AS line '
                 'MERGE (p1:Person {id: line["id"], name: line["name"]}) '
                 'MERGE (p2:Person {id: line["married"]}) '
                 'MERGE (p1)-[:Married]->(p2) '
                 'RETURN p1, p2'
    }
]

_run_transaction(statements)

[[<Node id=23 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>, <Node id=24 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>], [<Node id=24 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>, <Node id=23 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>]]
[<Node id=23 labels={'Person'} properties={'name': 'Alvin', 'id': '1'}>, <Node id=24 labels={'Person'} properties={'name': 'Emma', 'id': '2'}>]
[]
